# Double Deep Q-Network

In 2016, Google DeepMind ([Link](https://www.aaai.org/ocs/index.php/AAAI/AAAI16/paper/download/12389/11847)) decided to alter the DQN algorithm the same way the original Q-Learner algorithm was updated by adding a second network. The team found that the overestimation that affected the q-learner also affected the DQN algoritm. When the original double q-learner was introduced they proved that it worked in that setting and in this paper they prove it can be generalized to work with large scale function approximation.

**Reference**  
Van Hasselt, H., Guez, A., & Silver, D. (2016, February). Deep Reinforcement Learning with Double Q-Learning. In AAAI (Vol. 2, p. 5)